# 0.0 Imports

In [37]:
import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## 0.1 Help Functions

In [38]:
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)

## 0.2 Import dataset

In [39]:
df = pickle.load(open('../src/data/processed/df01.pkl', 'rb'))
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,currency,...,has_online_delivery,is_delivering_now,price_range,aggregate_rating,votes,name_country,custo_para_dois_corrigido,avalicao_texto,cuisines,nome_cor
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.01,14.45,Botswana Pula(P),...,0,0,3,4.60,619,Philippines,400.00,Excelente,Italian,Verde Escuro
1,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.02,14.56,Botswana Pula(P),...,0,0,4,4.70,469,Philippines,1127.27,Excelente,European,Verde Escuro
2,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.02,14.55,Botswana Pula(P),...,0,0,3,4.70,930,Philippines,254.55,Excelente,Filipino,Verde Escuro
3,18409457,Soban K-Town Grill,162,Makati City,"Level 3, Greenbelt 3, Ayala Center, Greenbelt,...","Greenbelt 3, San Lorenzo, Makati City","Greenbelt 3, San Lorenzo, Makati City, Makati ...",121.02,14.55,Botswana Pula(P),...,0,0,3,4.70,935,Philippines,472.73,Excelente,Korean,Verde Escuro
4,6318945,Nikkei,162,Makati City,"Unit GO3, Frabelle Business Center, 111 Rada S...",Legaspi Village,"Legaspi Village, Makati City",121.02,14.56,Botswana Pula(P),...,0,0,4,4.60,394,Philippines,581.82,Excelente,Japanese,Verde Escuro


# 1.0 Perguntas de negócio

In [40]:
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'currency',
       'has_table_booking', 'has_online_delivery', 'is_delivering_now',
       'price_range', 'aggregate_rating', 'votes', 'name_country',
       'custo_para_dois_corrigido', 'avalicao_texto', 'cuisines', 'nome_cor'],
      dtype='object')

## 1.1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [41]:
rest_votes = df[['restaurant_name','votes']].sort_values(by='votes', ascending=False).reset_index()
rest_votes.iloc[0,1]

'Bawarchi'

## 1.2. Qual o nome do restaurante com a maior nota média?

In [42]:
rest_nota = df[['restaurant_name','aggregate_rating']].sort_values(by='aggregate_rating', ascending=False).reset_index()
rest_nota.iloc[0,1]

'Barbeque Nation'

## 1.3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [43]:
rest_prato_dois = df[['restaurant_name','custo_para_dois_corrigido']].sort_values(by='custo_para_dois_corrigido', ascending=False).reset_index()
rest_prato_dois.iloc[0,1]

'Corner House'

## 1.4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [44]:
rest_brasil_menor_avaliac = df.loc[df['cuisines'] == 'Brazilian',['restaurant_name','aggregate_rating']].sort_values(by='aggregate_rating', ascending=True).reset_index()
rest_brasil_menor_avaliac.iloc[0,1]

'Tempero das Gerais'

## 1.5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [45]:
rest_brasil_maior_avaliac = df.loc[(df['cuisines'] == 'Brazilian') & (df['name_country'] == 'Brazil'), ['restaurant_name','aggregate_rating']].sort_values(by='aggregate_rating', ascending=True).reset_index()
rest_brasil_menor_avaliac.iloc[0,1]

'Tempero das Gerais'

## 1.6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [46]:
rest_online_avaliacoes = df.loc[:,['has_online_delivery','votes']].groupby('has_online_delivery').mean().reset_index()

if rest_online_avaliacoes.iloc[0,1] > rest_online_avaliacoes.iloc[0,0]:
    print(f'Os restaurantes com resultato {rest_online_avaliacoes.iloc[1,0]}, possuem {rest_online_avaliacoes.iloc[1,1]} votos na média')

Os restaurantes com resultato 1, possuem 838.8216639209226 votos na média


## 1.7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [51]:
rest_reserva_prato = df.loc[:,['has_table_booking','custo_para_dois_corrigido']].groupby('has_table_booking').mean().reset_index()
rest_reserva_prato

,has_table_booking,custo_para_dois_corrigido
0,0,144.57
1,1,177.38


## 1.8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [70]:
rest_bbq = df.loc[((df['cuisines'] == 'Japanese') | (df['cuisines'] == 'BBQ')) & (df['name_country'] == 'United States of America'),['name_country','cuisines','custo_para_dois_corrigido']].groupby(['name_country','cuisines']).mean().reset_index()
rest_bbq

,name_country,cuisines,custo_para_dois_corrigido
0,United States of America,BBQ,198.21
1,United States of America,Japanese,282.03
